# Create Emotion label

This notebook will create a label on the full "trumptweets.csv" data set. 
The text of Donald Trump´s tweets is going to be emotionally labeled. 
Therefore the code of the notebook "Label Test.ipynb" will be used. 
This notebook was been used to create a test set of data, which I used for some testing of visualization and classification. 


## Imports

In [1]:
# libraries Im going to use
import pandas as pd
from IPython.display import Image
import os
from langdetect import detect
import time
import os
from ibm_watson import ApiException

## Functions 

In [2]:
### Emotino classification function
# function to classify text (tweet, one tweet)
def emotion_classification(Text):
    """
    This function runs the Watson Natural Language Understanding API
    
    Input:
    - Text, which is going to be classifed into specific emotion
    
    Output:
    - list
    -- most important - emotion of the document
    """
    response = natural_language_understanding.analyze(
        text= Text, # inpute of the function
        features=Features(
            emotion=EmotionOptions(document=True),
            entities=EntitiesOptions(emotion=True, sentiment=True, limit=2),
            keywords=KeywordsOptions(emotion=True, sentiment=True,
                                     limit=2))
    ).get_result()
    return(response)

In [3]:
### get max emotion
def get_max_emotion(emotions_dict):
    """
    This function runs the classification/emotion with it´s given probability
    
    Input:
    - emotions_dict
    
    Output:
    - emotion_max_probability
    -- just returns the emotion with the highest probability
    - max_probability
    -- the probability the emotion come along with
    """
    
    # find max value and label
    max_probability = max(emotions_dict.values())  

    # getting all emotions containing the "max_probability" as string
    emotion_max_probability = [k for k, v in emotions_dict.items() if v == max_probability][0] 
    
    # return
    return(emotion_max_probability,max_probability)


In [4]:
### drop rows with pictures and links at the beginning of the tweet
def drop_pictures_links(df):
    """
    This function does removes the tweets (whole row), which start with a picture or link.
    Reason: Often it´s just the picture or the link, this causes a problem by analyzing the "text"
    ALSO: check the language of the tweet, if it´s not englisch the tweet will be removed
    
    Input:
    - df: the data frame with all unprocessed data
    
    Output:
    - df: without the rows which start with a picture or link
    - drop_list: list of the row index which got dropped
    """
    
    print("The uncleand dataframe has ", df.shape[0], " rows and ", df.shape[1], " columns.")
    # init list of all the rows which are going to be dropped
    drop_list = []
    
    # init values for the loop
    # check if tweets starts with picture or link
    start_picture_tweet = "pic.twitter.com"
    start_picture_tweet_length = len(start_picture_tweet)
    start_link_tweet = "https://"
    start_link_tweet_length = len(start_link_tweet)
    start_hashtag = "#"
    start_hashtag_length = len(start_hashtag)
    
    
    print("The following index and items are going to be removed:")
    # run throguh every row and check which rows start with a picture or link
    #for i in range(df.shape[0]):
    for i in df.index:
        
        # check language
        #if detect(df.text[i][0:start_link_tweet_length]) != "en": 
        #if(detect(data.text[i]) == "ar" or detect(data.text[i]) == "he" or detect(data.text[i]) == "fa"):
        #    drop_list.append(i)
        #    print(i, "lang", detect(data.text[i]), data.text[i])
        # check picture
        if df.text[i][0:start_picture_tweet_length] == start_picture_tweet:
            drop_list.append(i)
            print(i, "pic")
            
        # check for link
        if df.text[i][0:start_link_tweet_length] == start_link_tweet:
            drop_list.append(i)
            print(i, "link")
        # check for #
        if df.text[i][0:start_hashtag_length] == start_hashtag:
            drop_list.append(i)
            print(i, "#")
        
        
     
    # drop rows which start with pic or link
    df_new = df.drop(drop_list)
    
    print("The cleand dataframe has ", df_new.shape[0], " rows and ", df_new.shape[1], " columns.")
    # return list of rows to drop
    return(df_new, drop_list)

### Remove links at the end of the tweet

In [5]:
def remove_link(tweet):
    # remove the link at the end of the text
    if "https://" in tweet:
        # setting the maxsplit parameter to 1, will return a list with 2 elements!
        # access just the first element
        new_string = tweet.split("https://",1)[0]
        return(new_string)
    else:
        return(tweet)
def remove_pic(tweet):
    if "pic.twitter." in tweet:
        # setting the maxsplit parameter to 1, will return a list with 2 elements!
        # access just the first element
        new_string = tweet.split("pic.twitter",1)[0]
        return(new_string)
    else:
        return(tweet)
def remove_hashtag(tweet):
    if "#" in tweet:
        # setting the maxsplit parameter to 1, will return a list with 2 elements!
        # access just the first element
        new_string = tweet.split("#",1)[0]
        return(new_string)
    else:
        return(tweet)

# Check Tweets - picture, link and non-english

**Links** - just the beginning

In [6]:
def check_tweets_function(data):
    # init values for the loop
    # check if tweets starts with picture or link
    start_picture_tweet = "pic.twitter.com"
    start_picture_tweet_length = len(start_picture_tweet)
    start_link_tweet = "https://"
    start_link_tweet_length = len(start_link_tweet)

    # init variables for picture and link case
    index_pictures = []
    index_link = []

    # init variables for the non-english case
    amount_english_tweets = 0
    amount_non_english_tweets = 0
    index_non_english_tweets = []

    for i in range(len(data.text)):
        if(i == 1398): # very ugly way, don´t case
            i = i +1
        # check picture
        if data.text[i][0:start_picture_tweet_length] == start_picture_tweet:
            index_pictures.append(i)
        # check for link
        if data.text[i][0:start_link_tweet_length] == start_link_tweet:
            index_link.append(i)
        # remove if case tweet is Arabic, Hebrew or Persian
        #if(detect(data.text[i]) == "ar" or detect(data.text[i]) == "he" or detect(data.text[i]) == "fa"): 
        #    index_non_english_tweets.append(i)
    
    # show results
    print("Amount pictures:", len(index_pictures))
    print("Amount links:", len(index_link))
    #print("Non-english:", len(index_non_english_tweets))
    # results
    return({"index_picture": index_pictures,
           "index_links": index_link,
           #"index_non_english":index_non_english_tweets
           })

### Main function

In [7]:
def create_emotion1(df):
    """
    The function does transform the data frame, so that just important information are going to be used. 
    It also adds the emotion and probability of the emotion as column.
    
    Input:
    - df: this is going to be the data frame as we get it from the GitHub function
    
    Output:
    - df_new: data frame we important information, plus enmotion and emotion probability
    """
    
    # tranformation of the data frame
    # data frame just with relevant informantion (username, date, retweets, favorites)
    trump_tweets = df.iloc[:,0:5]

    # drop rows of the data frame which start with picture or link
    trump_tweets, rowindex_dropped = drop_pictures_links(trump_tweets) # --- function
    # reindex data frame - easier to run the loop
    trump_tweets = trump_tweets.reset_index()
    # init emotion and its probabilites as list - save the results  - going to be return values
    emotions_list = []
    emotions_prob_list = []
    
    # loop over every sentence in the data frame
    ##########################################################################################
    # Here happens the magic
    ##########################################################################################
    # This loop will take very long for the whole data set
    for i in range(trump_tweets.shape[0]):
        
        # print after every 100 classification to see process
        if i in range(0,trump_tweets.shape[0],10):
            print(i, "emotional classifications done!")

        # run the function with Watson API 
        
        # clean the tweet, remove link, picture and hashtag at the end 
        clean_tweet = remove_pic(remove_link(remove_hashtag(trump_tweets.text[i])))
        
        # the main function - create emotions of the cleaned text
        emotions = emotion_classification(clean_tweet) # --- function

        ### --- flag - start ---
        # check if there does a emotion exist
        # else classify with "emotionless, 0,0%"
        if("emotion" in emotions.keys()):
        ### --- flag - end ---

            # access just the emotion of the document (of the tweet)
            emotions_dict = emotions["emotion"]["document"]["emotion"]

            # get the max emotion value
            # get_max_emotion(emotions_dict) 
            emotion_max_probability,max_probability = get_max_emotion(emotions_dict) # --- function 

            # save results for each step
            emotions_list.append(emotion_max_probability)
            emotions_prob_list.append(max_probability)

        # classify emotion with "emotionsless"
        else:
            emotions_list.append("emotionless")
            emotions_prob_list.append(0.0)
    ##########################################################################################       
    # assign emotions and its probabilites to data frame
    trump_tweets = trump_tweets.assign(emotion = emotions_list)
    trump_tweets = trump_tweets.assign(emotion_probability = emotions_prob_list)
    
    # return new data frame
    return(trump_tweets)

In [8]:
def create_emotion(df):
    """
    The function does transform the data frame, so that just important information are going to be used. 
    It also adds the emotion and probability of the emotion as column.
    
    Input:
    - df: this is going to be the data frame as we get it from the GitHub function
    
    Output:
    - df_new: data frame we important information, plus enmotion and emotion probability
    -- column emotion: includes emotions & NaN values
    """
    
    # tranformation of the data frame
    # data frame just with relevant informantion (username, date, retweets, favorites)
    trump_tweets = df.loc[:,["username", "date", "retweets", "favorites", "text"]]

    # drop rows of the data frame which start with picture or link
    trump_tweets, rowindex_dropped = drop_pictures_links(trump_tweets) # --- function
    # reindex data frame - easier to run the loop
    trump_tweets = trump_tweets.reset_index()
    
    # init emotion and its probabilites as list - save the results  - going to be return values
    emotions_list = []
    emotions_list_index = []
    emotions_prob_list = []
    # Init: for the fail case
    fail_index = []
    fail_message = []
    
    # loop over every sentence in the data frame
    ##########################################################################################
    # Here happens the magic
    print("-- Wation Natural Language Understanding is now going to create emotions for every tweet--")
    ##########################################################################################
    # This loop will take very long for the whole data set
    for i in range(trump_tweets.shape[0]):
        
        # print after every 100 classification to see process
        if i in range(0,trump_tweets.shape[0],10):
            print(i, "emotional classifications done!")

        # run the function with Watson API 
        
        # clean the tweet, remove link, picture and hashtag at the end 
        clean_tweet = remove_pic(remove_link(remove_hashtag(trump_tweets.text[i])))
        
        # the main function - create emotions of the cleaned text
        try:
            emotions = emotion_classification(clean_tweet) # --- function
            emotions_flag = "NO_FAIL"
        except ApiException as ex: 
            print("FAIL:", i, clean_tweet)
            # Classify NAN emotion + save fail message
            emotions_list.append("NaN")
            emotions_prob_list.append(1.0)
            emotions_flag = "FAIL"
            fail_message.append(ex.message)
        
        ### --- flag - start ---
        # check if: the classification failed 
        # if not, continue, if yes, do next tweet
        if emotions_flag == "NO_FAIL":
            # check if there does a emotion exist
            # else classify with "emotionless, 0,0%"
            if("emotion" in emotions.keys()):
            ### --- flag - end ---

                # access just the emotion of the document (of the tweet)
                emotions_dict = emotions["emotion"]["document"]["emotion"]

                # get the max emotion value
                # get_max_emotion(emotions_dict) 
                emotion_max_probability,max_probability = get_max_emotion(emotions_dict) # --- function 

                # save results for each step
                emotions_list.append(emotion_max_probability)
                emotions_list_index.append(i)
                emotions_prob_list.append(max_probability)

            # classify emotion with "emotionsless"
            else:
                emotions_list.append("emotionless")
                emotions_list_index.append(i)
                emotions_prob_list.append(0.0)
    
    # give information about error messages
    for f_message in pd.unique(fail_message):
        print(f_message, ":", fail_message.count(f_message))
    
    ########################################################################################## 
    #return(trump_tweets,emotions_list,emotions_prob_list)
    # assign emotions and its probabilites to data frame
    trump_tweets = trump_tweets.assign(emotion = emotions_list)
    trump_tweets = trump_tweets.assign(emotion_probability = emotions_prob_list)
    
    # return new data frame
    return(trump_tweets)

## Set up of the API

In [9]:
### Liberaries
# libraries I need to import
import json

# if import does not work
# pip install --upgrade "ibm-watson>=4.2.1"
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions, EmotionOptions

In [10]:
### API set up
# I saved all my important data into a json file
# this file I open here to save the apikey and url in a variable
# since other people should not see my keys
with open('watson_api.json') as json_file:
    # save data in dict
    api_access = json.load(json_file)

# init variables neeeded 
apikey = api_access["apikey"]
url = api_access["url"]

In [11]:
### API settings
# settings for the api 
authenticator = IAMAuthenticator(apikey)
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=authenticator)

natural_language_understanding.set_service_url(url)

## The data

In [12]:
# load the tweets
data = pd.read_csv("trumptweets.csv", sep=';')

### Check the data


In [13]:
data.head()

,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink
0,realDonaldTrump,09.02.20 00:47,13459,72445,A great coach and a fantastic guy. His endorse...,NaN,NaN,NaN,"1,22629E+18",https://twitter.com/realDonaldTrump/status/122...
1,realDonaldTrump,08.02.20 22:08,47880,215503,Pete Rose played Major League Baseball for 24 ...,NaN,NaN,NaN,"1,22625E+18",https://twitter.com/realDonaldTrump/status/122...
2,realDonaldTrump,08.02.20 20:48,9452,37402,Total and complete Endorsement for Debbie Lesk...,NaN,#NAME?,NaN,"1,22623E+18",https://twitter.com/realDonaldTrump/status/122...
3,realDonaldTrump,08.02.20 20:40,17545,62484,Governor Cuomo wanted to see me this weekend. ...,NaN,NaN,NaN,"1,22623E+18",https://twitter.com/realDonaldTrump/status/122...
4,realDonaldTrump,08.02.20 20:01,27437,120598,We will not be touching your Social Security o...,NaN,NaN,NaN,"1,22622E+18",https://twitter.com/realDonaldTrump/status/122...


In [14]:
data.tail()

,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink
10245,realDonaldTrump,07.01.17 16:02,24681,87739,Having a good relationship with Russia is a go...,NaN,NaN,NaN,"8,17748E+17",https://twitter.com/realDonaldTrump/status/817...
10246,realDonaldTrump,07.01.17 13:03,16601,73661,Only reason the hacking of the poorly defended...,NaN,NaN,NaN,"8,17703E+17",https://twitter.com/realDonaldTrump/status/817...
10247,realDonaldTrump,07.01.17 12:56,15401,60280,Intelligence stated very strongly there was ab...,NaN,NaN,NaN,"8,17701E+17",https://twitter.com/realDonaldTrump/status/817...
10248,realDonaldTrump,07.01.17 04:53,13961,59218,Gross negligence by the Democratic National Co...,NaN,NaN,NaN,"8,1758E+17",https://twitter.com/realDonaldTrump/status/817...
10249,realDonaldTrump,07.01.17 01:07,6657,42476,Happy Birthday @EricTrump ! https://www. faceb...,NaN,#NAME?,NaN,"8,17523E+17",https://twitter.com/realDonaldTrump/status/817...


In [15]:
data.shape

(10250, 10)

In [43]:
data.iloc[48].text

'#SOTU2020 pic.twitter.com/W03gQLkdpo'

### Test data

one more small test before I run the whole data set!

In [16]:
# one more small test 
# create a test set of 200 tweets
data_test = data.iloc[0:400,:]
data_test

,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink
0,realDonaldTrump,09.02.20 00:47,13459,72445,A great coach and a fantastic guy. His endorse...,NaN,NaN,NaN,"1,22629E+18",https://twitter.com/realDonaldTrump/status/122...
1,realDonaldTrump,08.02.20 22:08,47880,215503,Pete Rose played Major League Baseball for 24 ...,NaN,NaN,NaN,"1,22625E+18",https://twitter.com/realDonaldTrump/status/122...
2,realDonaldTrump,08.02.20 20:48,9452,37402,Total and complete Endorsement for Debbie Lesk...,NaN,#NAME?,NaN,"1,22623E+18",https://twitter.com/realDonaldTrump/status/122...
3,realDonaldTrump,08.02.20 20:40,17545,62484,Governor Cuomo wanted to see me this weekend. ...,NaN,NaN,NaN,"1,22623E+18",https://twitter.com/realDonaldTrump/status/122...
4,realDonaldTrump,08.02.20 20:01,27437,120598,We will not be touching your Social Security o...,NaN,NaN,NaN,"1,22622E+18",https://twitter.com/realDonaldTrump/status/122...
...,...,...,...,...,...,...,...,...,...,...
395,realDonaldTrump,11.01.20 00:32,15163,90211,Will be interviewed tonight by Laura @Ingraham...,NaN,#NAME?,NaN,"1,21578E+18",https://twitter.com/realDonaldTrump/status/121...
396,realDonaldTrump,10.01.20 16:37,24488,120462,She will go down as perhaps the least successf...,NaN,NaN,NaN,"1,21566E+18",https://twitter.com/realDonaldTrump/status/121...
397,realDonaldTrump,10.01.20 16:31,18077,78224,I love constantly proving them wrong. It’s eas...,NaN,NaN,NaN,"1,21566E+18",https://twitter.com/realDonaldTrump/status/121...
398,realDonaldTrump,10.01.20 16:11,16815,86594,“I’ve been doing this for 40 years and I’ve ne...,NaN,#NAME?,NaN,"1,21565E+18",https://twitter.com/realDonaldTrump/status/121...


# Check Tweets test - picture, link and non-english

**Links** - just the beginning

In [17]:
check_tweet_test = check_tweets_function(data_test)

Amount pictures: 26
Amount links: 7


In [18]:
print("Totel items:", sum(map(len, check_tweet_test.values())))

Totel items: 33


# Make emotional classification - Test set

In [91]:
# create data frame 
start_time = time.time()
data_emotion = create_emotion(data_test)
print("--- %s seconds ---" % (time.time() - start_time))

The uncleand dataframe has  400  rows and  5  columns.
The following index and items are going to be removed:
13 pic
23 pic
34 pic
38 link
41 pic
43 pic
44 pic
48 #
49 pic
58 pic
105 pic
106 pic
107 pic
108 pic
111 pic
112 pic
117 pic
139 link
142 lang ar هذا ما قد تبدو عليه دولة فلسطين المستقبلية بعاصمة في أجزاء من القدس الشرقية. pic.twitter.com/CFuYwwjSso
144 lang he תמיד אעמוד לצד מדינת ישראל והעם היהודי. אני תומך בחוזקה בבטיחותם ובטחונם ובזכותם לחיות במולדתם ההיסטורית. הגיע הזמן לשלום! pic.twitter.com/AtNnQtnGZs
145 link
156 link
157 pic
167 pic
187 pic
189 lang fa وزیر امور خارجه ایران می گوید ایران خواستار مذاکره با ایالات متحده است اما می خواهد که تحریم ها برداشته شود. @FoxNews @OANN نه، مرسی! https:// twitter.com/realDonaldTrum p/status/1221225245220265985 …
205 link
208 #
222 pic
223 pic
225 pic
278 link
294 pic
307 lang fa مردم نجیب ایران، که آمریکا را دوست می دارند، سزاوار دولتی هستند که بیش از تمرکز بر کشتن آنها به جرم احترام خواهی، به آنها کمک کند تا به رؤیاهایشان دست یابن

ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: c841d487-daf2-49d2-86fe-38743ef993a8


FAIL: 12 Not good! 
20 emotional classifications done!


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: 2cb36be1-2ae6-4a03-b254-ccffe03a3bff


FAIL: 25 So do I! 
30 emotional classifications done!


...

FAIL: 252 USA! USA! USA!
260 emotional classifications done!
270 emotional classifications done!
280 emotional classifications done!


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: a4fb331b-7708-4b69-9792-aa17e33d7af5


290 emotional classifications done!
FAIL: 290 THANK YOU! 
300 emotional classifications done!
310 emotional classifications done!


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: 10ca34ac-afdb-436b-988e-569d8e63730e


FAIL: 312 Thank you! 
320 emotional classifications done!
330 emotional classifications done!
340 emotional classifications done!
350 emotional classifications done!
not enough text for language id : 15
unsupported text language: unknown : 1
--- 150.5794951915741 seconds ---


In [93]:
data_emotion

,index,username,date,retweets,favorites,text,emotion,emotion_probability
0,0,realDonaldTrump,09.02.20 00:47,13459,72445,A great coach and a fantastic guy. His endorse...,joy,0.887483
1,1,realDonaldTrump,08.02.20 22:08,47880,215503,Pete Rose played Major League Baseball for 24 ...,sadness,0.320811
2,2,realDonaldTrump,08.02.20 20:48,9452,37402,Total and complete Endorsement for Debbie Lesk...,joy,0.755680
3,3,realDonaldTrump,08.02.20 20:40,17545,62484,Governor Cuomo wanted to see me this weekend. ...,sadness,0.491409
4,4,realDonaldTrump,08.02.20 20:01,27437,120598,We will not be touching your Social Security o...,anger,0.648076
...,...,...,...,...,...,...,...,...
352,395,realDonaldTrump,11.01.20 00:32,15163,90211,Will be interviewed tonight by Laura @Ingraham...,joy,0.841806
353,396,realDonaldTrump,10.01.20 16:37,24488,120462,She will go down as perhaps the least successf...,joy,0.563260
354,397,realDonaldTrump,10.01.20 16:31,18077,78224,I love constantly proving them wrong. It’s eas...,sadness,0.405583
355,398,realDonaldTrump,10.01.20 16:11,16815,86594,“I’ve been doing this for 40 years and I’ve ne...,joy,0.421897


# Make emotional classification - Full data set

### Check how long the cell runs to make all classifications

In [108]:
data[1386:1400]

,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink
1386,realDonaldTrump,27.10.19 15:31,24652,92817,https://www. pscp.tv/w/cIWhpTFvTlFs TFJub1dwUX...,NaN,NaN,NaN,"1,18846E+18",https://twitter.com/realDonaldTrump/status/118...
1387,realDonaldTrump,27.10.19 11:21,27288,131951,"As Diwali commences, @FLOTUS Melania and I wis...",NaN,#NAME?,#HappyDiwali,"1,1884E+18",https://twitter.com/realDonaldTrump/status/118...
1388,realDonaldTrump,27.10.19 02:23,71808,360486,Something very big has just happened!,NaN,NaN,NaN,"1,18826E+18",https://twitter.com/realDonaldTrump/status/118...
1389,realDonaldTrump,26.10.19 22:26,11997,51307,....Matt has my Complete and Total Endorsement...,NaN,#NAME?,NaN,"1,18819E+18",https://twitter.com/realDonaldTrump/status/118...
1390,realDonaldTrump,26.10.19 22:26,14758,63159,Governor @MattBevin has done a wonderful job f...,NaN,#NAME?,NaN,"1,18819E+18",https://twitter.com/realDonaldTrump/status/118...
1391,realDonaldTrump,26.10.19 22:20,13118,53713,....He loves our Military and supports our Vet...,NaN,NaN,NaN,"1,18819E+18",https://twitter.com/realDonaldTrump/status/118...
1392,realDonaldTrump,26.10.19 22:20,18548,68447,MISSISSIPPI! There is a VERY important electio...,NaN,#NAME?,NaN,"1,18819E+18",https://twitter.com/realDonaldTrump/status/118...
1393,realDonaldTrump,26.10.19 22:17,14061,52052,....Our Republican candidate @EddieRispone is ...,NaN,#NAME?,NaN,"1,18819E+18",https://twitter.com/realDonaldTrump/status/118...
1394,realDonaldTrump,26.10.19 22:17,21459,74535,LOUISIANA! Extreme Democrat John Bel Edwards h...,NaN,NaN,NaN,"1,18819E+18",https://twitter.com/realDonaldTrump/status/118...
1395,realDonaldTrump,26.10.19 16:43,27135,130051,Where’s the Whistleblower?,NaN,NaN,NaN,"1,1881E+18",https://twitter.com/realDonaldTrump/status/118...


In [118]:
# Make emotional classification
# create data frame 
start_time = time.time()
new_data = create_emotion(data)
print("--- %s seconds ---" % (time.time() - start_time))

The uncleand dataframe has  10250  rows and  5  columns.
The following index and items are going to be removed:
13 pic
23 pic
34 pic
38 link
41 pic
43 pic
44 pic
48 #
49 pic
58 pic
105 pic
106 pic
107 pic
108 pic
111 pic
112 pic
117 pic
139 link
145 link
156 link
157 pic
167 pic
187 pic
205 link
208 #
222 pic
223 pic
225 pic
278 link
294 pic
333 link
361 pic
362 pic
363 pic
383 pic
431 pic
432 pic
445 pic
455 pic
465 pic
550 pic
551 pic
570 pic
571 pic
572 pic
574 link
579 pic
608 #
626 pic
653 pic
668 link
669 pic
708 pic
711 pic
712 pic
748 #
772 pic
820 pic
821 pic
854 pic
880 pic
915 link
923 pic
925 pic
928 pic
936 pic
939 pic
963 pic
964 pic
965 pic
977 pic
985 pic
987 pic
1009 pic
1013 link
1020 pic
1030 pic
1031 pic
1039 link
1045 pic
1048 pic
1049 pic
1051 pic
1055 pic
1060 pic
1061 pic
1062 pic
1064 pic
1065 pic
1066 pic
1067 pic
1079 pic
1080 pic
1081 pic
1091 #
1092 pic
1094 link
1099 pic
1100 pic
1101 pic
1102 pic
1103 pic
1126 pic
1164 link
1165 link
1168 link
1169 pic
11

ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: 80d70d99-7223-4633-bc74-4408e55a4b00


FAIL: 12 Not good! 
20 emotional classifications done!


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: 5d5c32d4-bb60-4c22-b8cb-2aec8fb903dd


FAIL: 25 So do I! 
30 emotional classifications done!


...

FAIL: 8555 Received a 
8560 emotional classifications done!


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: 2e160ac7-d382-4c9b-a544-ef0d96956d04


FAIL: 8563 As 
8570 emotional classifications done!
8580 emotional classifications done!
8590 emotional classifications done!
8600 emotional classifications done!
8610 emotional classifications done!
8620 emotional classifications done!
8630 emotional classifications done!
8640 emotional classifications done!
8650 emotional classifications done!


ERROR:root:Error in service call
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/urllib3/connection.py", line 160, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/urllib3/util/connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/socket.py", line 748, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 677, in urlopen
    chunked=chunked,
  File "/Users/phillipholscher/opt/anaconda3/lib/python

ConnectionError: HTTPSConnectionPool(host='api.eu-gb.natural-language-understanding.watson.cloud.ibm.com', port=443): Max retries exceeded with url: /instances/f80638d4-fc46-4e76-b427-8087219c13fb/v1/analyze?version=2019-07-12 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff6a5c2cc90>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

# Make emotional classification - split data

Not sure what the error message caused

- assumption 1: the many requests at ones -> split the data in smaller parts
- assumption 2: the 30k requests are all used (unlikely)

In [19]:
# one more small test 
# create a test set of 8000 tweets
data_2000 = data.iloc[0:2000,:]
data_4000 = data.iloc[2000:4000,:]
data_6000 = data.iloc[4000:6000,:]
data_8000 = data.iloc[6000:8000,:]
data_10000 = data.iloc[8000:,:]


## Split data - first part

# 0-2000

In [20]:
start_time = time.time()
new_data_2000 = create_emotion(data_2000)
print("--- %s seconds ---" % (time.time() - start_time))

The uncleand dataframe has  2000  rows and  5  columns.
The following index and items are going to be removed:
13 pic
23 pic
34 pic
38 link
41 pic
43 pic
44 pic
48 #
49 pic
58 pic
105 pic
106 pic
107 pic
108 pic
111 pic
112 pic
117 pic
139 link
145 link
156 link
157 pic
167 pic
187 pic
205 link
208 #
222 pic
223 pic
225 pic
278 link
294 pic
333 link
361 pic
362 pic
363 pic
383 pic
431 pic
432 pic
445 pic
455 pic
465 pic
550 pic
551 pic
570 pic
571 pic
572 pic
574 link
579 pic
608 #
626 pic
653 pic
668 link
669 pic
708 pic
711 pic
712 pic
748 #
772 pic
820 pic
821 pic
854 pic
880 pic
915 link
923 pic
925 pic
928 pic
936 pic
939 pic
963 pic
964 pic
965 pic
977 pic
985 pic
987 pic
1009 pic
1013 link
1020 pic
1030 pic
1031 pic
1039 link
1045 pic
1048 pic
1049 pic
1051 pic
1055 pic
1060 pic
1061 pic
1062 pic
1064 pic
1065 pic
1066 pic
1067 pic
1079 pic
1080 pic
1081 pic
1091 #
1092 pic
1094 link
1099 pic
1100 pic
1101 pic
1102 pic
1103 pic
1126 pic
1164 link
1165 link
1168 link
1169 pic
117

ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: 70c4fc58-5511-4830-b31e-5d62dca9bcea


FAIL: 12 Not good! 
20 emotional classifications done!


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: 11dec349-d864-4073-8d2a-23c241414dc8


FAIL: 25 So do I! 
30 emotional classifications done!


...

FAIL: 1581 So amazing! 
1590 emotional classifications done!
1600 emotional classifications done!
1610 emotional classifications done!
1620 emotional classifications done!
1630 emotional classifications done!
1640 emotional classifications done!
1650 emotional classifications done!
1660 emotional classifications done!


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: 56862e9a-06b6-45c5-84d4-048a04e63044


FAIL: 1663 Too bad! 
1670 emotional classifications done!
1680 emotional classifications done!
1690 emotional classifications done!
1700 emotional classifications done!
1710 emotional classifications done!
1720 emotional classifications done!
1730 emotional classifications done!
1740 emotional classifications done!
1750 emotional classifications done!


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: c9432a96-97fa-4c17-87e3-918c54eaa9e0


FAIL: 1756 THANK YOU! 
1760 emotional classifications done!
1770 emotional classifications done!
1780 emotional classifications done!
1790 emotional classifications done!
1800 emotional classifications done!
1810 emotional classifications done!
1820 emotional classifications done!
not enough text for language id : 65
unsupported text language: unknown : 1
unsupported text language: fa : 6
unsupported text language: ca : 2
--- 711.4210312366486 seconds ---


In [21]:
new_data_2000

NameError: name 'new_data_2000' is not defined

In [62]:
new_data_2000.to_csv("new_data_2000.csv", index = None, header = True)

# 2000-4000

In [40]:
start_time = time.time()
new_data_4000 = create_emotion(data_4000)
print("--- %s seconds ---" % (time.time() - start_time))

The uncleand dataframe has  2000  rows and  5  columns.
The following index and items are going to be removed:
2015 pic
2016 link
2020 pic
2021 link
2023 pic
2026 pic
2039 pic
2041 pic
2052 pic
2054 #
2060 pic
2064 pic
2067 link
2076 pic
2085 pic
2086 pic
2087 pic
2089 pic
2090 pic
2098 pic
2112 pic
2113 pic
2130 pic
2223 pic
2227 pic
2274 pic
2279 pic
2296 pic
2300 pic
2301 pic
2335 pic
2356 #
2368 link
2421 pic
2430 pic
2436 pic
2482 link
2488 pic
2584 pic
2634 pic
2635 pic
2643 link
2732 pic
2800 pic
2805 pic
2807 pic
2833 pic
2883 pic
2890 pic
2906 pic
2908 pic
2943 pic
2947 pic
2952 pic
2953 pic
2986 pic
3002 pic
3004 pic
3017 pic
3035 pic
3093 pic
3131 #
3174 pic
3175 pic
3227 pic
3229 #
3257 pic
3263 link
3281 pic
3287 pic
3308 pic
3323 pic
3350 pic
3351 pic
3356 pic
3361 pic
3368 pic
3424 pic
3440 pic
3508 #
3509 pic
3510 pic
3516 pic
3517 pic
3554 pic
3555 pic
3556 #
3559 #
3564 #
3568 #
3579 #
3584 link
3585 #
3637 pic
3664 pic
3666 #
3667 pic
3693 pic
3728 link
3871 pic
3873

ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: ae5169f3-55d7-464a-acc4-eedf2cc119b4


FAIL: 39 THANK YOU! 
40 emotional classifications done!
50 emotional classifications done!
60 emotional classifications done!
70 emotional classifications done!
80 emotional classifications done!
90 emotional classifications done!


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: 6ac7e0c7-e88c-412d-8550-8127585d2d41


FAIL: 91 Thank you! 
100 emotional classifications done!


...

FAIL: 1404 Thank you! 
1410 emotional classifications done!
1420 emotional classifications done!
1430 emotional classifications done!
1440 emotional classifications done!
1450 emotional classifications done!
1460 emotional classifications done!
1470 emotional classifications done!
1480 emotional classifications done!
1490 emotional classifications done!


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: fcbdd294-69c3-4597-90f9-ceea8a37e347


1500 emotional classifications done!
FAIL: 1500 Thank you! 
1510 emotional classifications done!
1520 emotional classifications done!
1530 emotional classifications done!
1540 emotional classifications done!
1550 emotional classifications done!
1560 emotional classifications done!
1570 emotional classifications done!
1580 emotional classifications done!
1590 emotional classifications done!
1600 emotional classifications done!
1610 emotional classifications done!
1620 emotional classifications done!
1630 emotional classifications done!
1640 emotional classifications done!
1650 emotional classifications done!
1660 emotional classifications done!
1670 emotional classifications done!
1680 emotional classifications done!
1690 emotional classifications done!
1700 emotional classifications done!
1710 emotional classifications done!
1720 emotional classifications done!
1730 emotional classifications done!
1740 emotional classifications done!
1750 emotional classifications done!
1760 emotional 

ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: 7467c9af-89fb-430a-bf91-62aefe837c25


FAIL: 1771 Really sad! 
1780 emotional classifications done!
1790 emotional classifications done!
1800 emotional classifications done!
1810 emotional classifications done!
1820 emotional classifications done!
1830 emotional classifications done!
1840 emotional classifications done!
1850 emotional classifications done!
1860 emotional classifications done!
1870 emotional classifications done!
1880 emotional classifications done!
1890 emotional classifications done!
not enough text for language id : 23
unsupported text language: ca : 1
upstream connect error or disconnect/reset before headers. reset reason: connection termination : 2
--- 756.6998200416565 seconds ---


In [63]:
new_data_4000.to_csv("new_data_4000.csv", index = None, header = True)

In [41]:
new_data_4000

,index,username,date,retweets,favorites,text,emotion,emotion_probability
0,2000,realDonaldTrump,27.09.19 01:02,23099,96945,The President of Ukraine said that he was NOT ...,joy,0.360345
1,2001,realDonaldTrump,27.09.19 01:01,13271,55921,No one has done more behind the scenes for STR...,joy,0.598367
2,2002,realDonaldTrump,27.09.19 00:58,9586,45362,"Congratulations to my friend @SenShelby , our ...",joy,0.799805
3,2003,realDonaldTrump,26.09.19 20:13,29786,119288,"Liddle’ Adam Schiff, who has worked unsuccessf...",sadness,0.549624
4,2004,realDonaldTrump,26.09.19 18:48,18882,86062,Adam Schiff has zero credibility. Another fant...,sadness,0.559756
...,...,...,...,...,...,...,...,...
1893,3995,realDonaldTrump,01.05.19 13:01,15464,70659,Gallup Poll: 56% of Americans rate their finan...,joy,0.562910
1894,3996,realDonaldTrump,01.05.19 12:50,20850,105698,I am overriding the Decommission Order of the ...,joy,0.482437
1895,3997,realDonaldTrump,01.05.19 12:36,19323,78162,“The Democrats can’t come to grips with the fa...,disgust,0.411829
1896,3998,realDonaldTrump,01.05.19 11:56,13312,60876,I’ve done more for Firefighters than this dues...,anger,0.577340


# 4000-6000

In [20]:
start_time = time.time()
new_data_6000 = create_emotion(data_6000)
print("--- %s seconds ---" % (time.time() - start_time))

The uncleand dataframe has  2000  rows and  5  columns.
The following index and items are going to be removed:
4110 pic
4114 pic
4115 pic
4134 pic
4222 pic
4290 pic
4334 pic
4335 pic
4343 pic
4344 pic
4348 pic
4351 pic
4352 pic
4363 link
4364 pic
4376 pic
4377 pic
4379 pic
4386 pic
4388 pic
4400 pic
4406 pic
4407 pic
4410 pic
4413 pic
4427 pic
4442 pic
4444 pic
4446 pic
4447 pic
4449 pic
4459 link
4475 pic
4508 pic
4568 link
4587 pic
4588 pic
4654 pic
4679 pic
4680 pic
4690 pic
4691 pic
4700 link
4701 pic
4714 pic
4755 pic
4756 #
4781 pic
4817 #
4835 link
4884 pic
4895 link
4904 pic
4907 pic
4935 pic
4955 pic
4995 link
5012 pic
5036 pic
5038 pic
5260 #
5314 #
5336 #
5398 pic
5578 pic
5593 #
5640 #
5686 #
5687 pic
5696 link
5730 #
5735 #
5757 pic
5758 pic
5759 pic
5760 pic
5807 pic
5816 pic
5825 pic
5826 pic
5827 pic
5839 link
5850 pic
5851 link
5889 pic
5907 link
5917 link
5922 pic
5938 link
5944 pic
5956 pic
5957 pic
5963 pic
5977 pic
5979 pic
5990 pic
The cleand dataframe has  1904  

ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: 2b98be75-3752-4d74-bc0c-1930e8d9761c


FAIL: 44 Thank you! 
50 emotional classifications done!
60 emotional classifications done!
70 emotional classifications done!
80 emotional classifications done!
90 emotional classifications done!
100 emotional classifications done!
110 emotional classifications done!
120 emotional classifications done!
130 emotional classifications done!
140 emotional classifications done!
150 emotional classifications done!
160 emotional classifications done!
170 emotional classifications done!
180 emotional classifications done!
190 emotional classifications done!
200 emotional classifications done!
210 emotional classifications done!
220 emotional classifications done!
230 emotional classifications done!
240 emotional classifications done!
250 emotional classifications done!
260 emotional classifications done!
270 emotional classifications done!
280 emotional classifications done!
290 emotional classifications done!
300 emotional classifications done!
310 emotional classifications done!
320 emotiona

ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: bfb766e2-5345-445f-92ce-46cfa63a0437


FAIL: 383 A total loser! 
390 emotional classifications done!
400 emotional classifications done!
410 emotional classifications done!
420 emotional classifications done!


...

FAIL: 1513 Fantastic 
1520 emotional classifications done!
1530 emotional classifications done!
1540 emotional classifications done!
1550 emotional classifications done!


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: 946f3bdb-4d93-455c-9273-fd976fe87dde


FAIL: 1554 So Revealing! 
1560 emotional classifications done!
1570 emotional classifications done!
1580 emotional classifications done!
1590 emotional classifications done!
1600 emotional classifications done!
1610 emotional classifications done!
1620 emotional classifications done!
1630 emotional classifications done!
1640 emotional classifications done!
1650 emotional classifications done!
1660 emotional classifications done!
1670 emotional classifications done!
1680 emotional classifications done!
1690 emotional classifications done!
1700 emotional classifications done!
1710 emotional classifications done!
1720 emotional classifications done!
1730 emotional classifications done!


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: 698b9e03-6399-49f1-938f-ed92db01badc


FAIL: 1739 Happy 
1740 emotional classifications done!
1750 emotional classifications done!
1760 emotional classifications done!
1770 emotional classifications done!
1780 emotional classifications done!
1790 emotional classifications done!
1800 emotional classifications done!
1810 emotional classifications done!
1820 emotional classifications done!
1830 emotional classifications done!
1840 emotional classifications done!
1850 emotional classifications done!
1860 emotional classifications done!
1870 emotional classifications done!
1880 emotional classifications done!
1890 emotional classifications done!
1900 emotional classifications done!
not enough text for language id : 11
unsupported text language: fa : 1
--- 766.2029981613159 seconds ---


In [21]:
new_data_6000.to_csv("new_data_6000.csv", index = None, header = True)

In [22]:
new_data_6000

,index,username,date,retweets,favorites,text,emotion,emotion_probability
0,4000,realDonaldTrump,30.04.19 23:09,29015,108449,"....embargo, together with highest-level sanct...",joy,0.522075
1,4001,realDonaldTrump,30.04.19 23:09,37660,124752,If Cuban Troops and Militia do not immediately...,sadness,0.456526
2,4002,realDonaldTrump,30.04.19 22:20,6514,34536,CONGRATULATIONS to the 2018 @NASCAR Cup Series...,joy,0.900825
3,4003,realDonaldTrump,30.04.19 20:23,50816,185488,I am monitoring the situation in Venezuela ver...,joy,0.375252
4,4004,realDonaldTrump,30.04.19 20:05,12492,57035,....up like a rocket if we did some lowering o...,joy,0.331070
...,...,...,...,...,...,...,...,...
1899,5995,realDonaldTrump,19.09.18 13:40,12892,59990,....it was impossible - and then already it’s ...,sadness,0.372713
1900,5996,realDonaldTrump,19.09.18 13:34,16506,71571,“The recovery got started on Election Day 2016...,sadness,0.838875
1901,5997,realDonaldTrump,19.09.18 06:11,14765,69649,"....returned home to the United States. Also, ...",joy,0.650108
1902,5998,realDonaldTrump,19.09.18 06:04,22506,92545,Kim Jong Un has agreed to allow Nuclear inspec...,joy,0.509397


# 6000-8000 

In [23]:
start_time = time.time()
new_data_8000 = create_emotion(data_8000)
print("--- %s seconds ---" % (time.time() - start_time))

The uncleand dataframe has  2000  rows and  5  columns.
The following index and items are going to be removed:
6000 pic
6014 pic
6063 #
6067 pic
6074 #
6077 #
6083 #
6085 pic
6192 pic
6219 #
6238 pic
6276 pic
6288 pic
6289 pic
6290 pic
6295 pic
6296 pic
6298 link
6299 pic
6321 pic
6338 pic
6342 pic
6350 pic
6409 pic
6516 #
6550 pic
6567 pic
6615 #
6623 #
6708 pic
6715 #
6767 pic
6914 link
6925 #
6998 pic
7061 pic
7081 #
7100 pic
7146 #
7254 #
7263 #
7322 #
7470 link
7542 #
7563 pic
7922 #
The cleand dataframe has  1954  rows and  5  columns.
-- Wation Natural Language Understanding is now going to create emotions for every tweet--
0 emotional classifications done!
10 emotional classifications done!
20 emotional classifications done!
30 emotional classifications done!
40 emotional classifications done!
50 emotional classifications done!
60 emotional classifications done!
70 emotional classifications done!
80 emotional classifications done!
90 emotional classifications done!
100 emotiona

ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: 48993fb6-bb25-42b2-b62e-b04207c4f0ef


FAIL: 128 TREASON?
130 emotional classifications done!


ERROR:root:unsupported text language: la
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: unsupported text language: la, Code: 400 , X-global-transaction-id: 128ad157-307c-4335-af47-3a8a3c18c8cc


FAIL: 138 “Secretary Mattis Nukes Woodward Allegations” 
140 emotional classifications done!
150 emotional classifications done!
160 emotional classifications done!
170 emotional classifications done!
180 emotional classifications done!
190 emotional classifications done!
200 emotional classifications done!
210 emotional classifications done!
220 emotional classifications done!
230 emotional classifications done!
240 emotional classifications done!
250 emotional classifications done!


...

FAIL: 1633 Happy 
1640 emotional classifications done!
1650 emotional classifications done!
1660 emotional classifications done!
1670 emotional classifications done!
1680 emotional classifications done!
1690 emotional classifications done!
1700 emotional classifications done!


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: fddee01e-457a-441d-9f0f-996173812136


1710 emotional classifications done!
FAIL: 1710 CPAC Today!
1720 emotional classifications done!
1730 emotional classifications done!
1740 emotional classifications done!
1750 emotional classifications done!
1760 emotional classifications done!
1770 emotional classifications done!
1780 emotional classifications done!
1790 emotional classifications done!
1800 emotional classifications done!
1810 emotional classifications done!
1820 emotional classifications done!
1830 emotional classifications done!
1840 emotional classifications done!
1850 emotional classifications done!
1860 emotional classifications done!
1870 emotional classifications done!
1880 emotional classifications done!
1890 emotional classifications done!
1900 emotional classifications done!
1910 emotional classifications done!
1920 emotional classifications done!


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: 80568e4a-6caa-42c4-83a2-dbaa8a98fb11


FAIL: 1924 AMERICA FIRST!
1930 emotional classifications done!
1940 emotional classifications done!
1950 emotional classifications done!
not enough text for language id : 16
unsupported text language: la : 1
unsupported text language: ca : 1
unsupported text language: unknown : 1
--- 794.0651609897614 seconds ---


In [24]:
new_data_8000.to_csv("new_data_8000.csv", index = None, header = True)

In [25]:
new_data_8000

,index,username,date,retweets,favorites,text,emotion,emotion_probability
0,6001,realDonaldTrump,18.09.18 23:20,8520,42649,"Today, it was my great honor to welcome @prezy...",joy,0.937340
1,6002,realDonaldTrump,18.09.18 20:21,15526,66556,"Today, I took action to strengthen our Nation’...",fear,0.551147
2,6003,realDonaldTrump,18.09.18 17:59,11342,56876,Thank you to our great Coast Guard for doing s...,joy,0.880532
3,6004,realDonaldTrump,18.09.18 17:50,9357,47101,"...that FEMA, our Military, and our First Resp...",joy,0.421596
4,6005,realDonaldTrump,18.09.18 17:50,13279,64890,"Right now, everybody is saying what a great jo...",anger,0.476538
...,...,...,...,...,...,...,...,...
1949,7995,realDonaldTrump,11.01.18 00:37,19243,79833,Cutting taxes and simplifying regulations make...,joy,0.843351
1950,7996,realDonaldTrump,11.01.18 00:07,13029,61231,The United States needs the security of the Wa...,fear,0.326405
1951,7997,realDonaldTrump,10.01.18 22:31,7891,44914,"Today, it was my great honor to welcome Prime ...",joy,0.823095
1952,7998,realDonaldTrump,10.01.18 21:08,9914,49906,I want to thank my @Cabinet for working tirele...,joy,0.826462


# 8000 - all

In [28]:
start_time = time.time()
new_data_10000 = create_emotion(data_10000)
print("--- %s seconds ---" % (time.time() - start_time))

The uncleand dataframe has  2250  rows and  5  columns.
The following index and items are going to be removed:
8403 #
8458 #
8497 #
8620 #
8675 #
8783 #
8812 #
8850 #
8900 #
9010 #
9092 #
9211 #
9243 #
9342 #
9345 #
9359 #
9396 #
9474 #
9514 #
9537 #
9538 #
9591 #
9595 #
9607 #
9612 #
9655 #
9678 #
9746 #
9749 #
9820 #
9874 #
9946 #
10021 #
The cleand dataframe has  2217  rows and  5  columns.
-- Wation Natural Language Understanding is now going to create emotions for every tweet--
0 emotional classifications done!
10 emotional classifications done!
20 emotional classifications done!
30 emotional classifications done!
40 emotional classifications done!
50 emotional classifications done!
60 emotional classifications done!
70 emotional classifications done!
80 emotional classifications done!
90 emotional classifications done!
100 emotional classifications done!
110 emotional classifications done!


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: 73a88032-8681-466b-806d-e7d785dee51c


FAIL: 114 So true! 
120 emotional classifications done!
130 emotional classifications done!
140 emotional classifications done!
150 emotional classifications done!
160 emotional classifications done!
170 emotional classifications done!
180 emotional classifications done!
190 emotional classifications done!
200 emotional classifications done!
210 emotional classifications done!
220 emotional classifications done!
230 emotional classifications done!
240 emotional classifications done!
250 emotional classifications done!
260 emotional classifications done!
270 emotional classifications done!
280 emotional classifications done!


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: dc5c01ff-3a94-47ac-8e71-4468d84f186b


FAIL: 282 Happy 
290 emotional classifications done!
300 emotional classifications done!
310 emotional classifications done!
320 emotional classifications done!
330 emotional classifications done!


...

FAIL: 1931 Going to CPAC!
1940 emotional classifications done!
1950 emotional classifications done!
1960 emotional classifications done!
1970 emotional classifications done!
1980 emotional classifications done!
1990 emotional classifications done!
2000 emotional classifications done!
2010 emotional classifications done!
2020 emotional classifications done!
2030 emotional classifications done!
2040 emotional classifications done!
2050 emotional classifications done!
2060 emotional classifications done!
2070 emotional classifications done!
2080 emotional classifications done!


ERROR:root:unsupported text language: la
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: unsupported text language: la, Code: 400 , X-global-transaction-id: a8818c6c-c48e-4c80-8c00-769054879d73


FAIL: 2083 Congratulations Secretary Mattis! 


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/Users/phillipholscher/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: b8e7fe74-a7c8-455c-be15-addcf3afa836


FAIL: 2086 The 
2090 emotional classifications done!
2100 emotional classifications done!
2110 emotional classifications done!
2120 emotional classifications done!
2130 emotional classifications done!
2140 emotional classifications done!
2150 emotional classifications done!
2160 emotional classifications done!
2170 emotional classifications done!
2180 emotional classifications done!
2190 emotional classifications done!
2200 emotional classifications done!
2210 emotional classifications done!
not enough text for language id : 23
unsupported text language: unknown : 1
unsupported text language: la : 1
--- 5143.8477737903595 seconds ---


In [29]:
new_data_10000.to_csv("new_data_10000.csv", index = None, header = True)

In [30]:
new_data_10000

,index,username,date,retweets,favorites,text,emotion,emotion_probability
0,8000,realDonaldTrump,10.01.18 16:00,18833,81336,"The fact that Sneaky Dianne Feinstein, who has...",anger,0.524542
1,8001,realDonaldTrump,10.01.18 15:11,21940,95371,It just shows everyone how broken and unfair o...,sadness,0.661137
2,8002,realDonaldTrump,10.01.18 05:23,11424,53239,". @ICEgov HSI agents and ERO officers, on beha...",joy,0.742836
3,8003,realDonaldTrump,10.01.18 04:48,10032,47160,Thank you @GOPLeader Kevin McCarthy! Couldn’t ...,joy,0.747622
4,8004,realDonaldTrump,10.01.18 01:16,20275,109383,"As I made very clear today, our country needs ...",fear,0.292884
...,...,...,...,...,...,...,...,...
2212,10245,realDonaldTrump,07.01.17 16:02,24681,87739,Having a good relationship with Russia is a go...,sadness,0.522362
2213,10246,realDonaldTrump,07.01.17 13:03,16601,73661,Only reason the hacking of the poorly defended...,sadness,0.567706
2214,10247,realDonaldTrump,07.01.17 12:56,15401,60280,Intelligence stated very strongly there was ab...,sadness,0.490432
2215,10248,realDonaldTrump,07.01.17 04:53,13961,59218,Gross negligence by the Democratic National Co...,disgust,0.746146


# Merge dataframes

In [ ]:
# one more small test 
# create a test set of 8000 tweets
data_2000 = data.iloc[0:2000,:]
data_4000 = data.iloc[2000:4000,:]
data_6000 = data.iloc[4000:6000,:]
data_8000 = data.iloc[6000:8000,:]
data_10000 = data.iloc[8000:,:]

In [32]:
# merge data frames
data_new = pd.concat([data2000,data4000,data_6000,data_8000,data_10000])

NameError: name 'data2000' is not defined



## Problem - Language
He post stuff in:
- Arabic
- Hebräisch

In [18]:
data_test.iloc[142,:]

username                                       realDonaldTrump
date                                            28.01.20 19:26
retweets                                                 11161
favorites                                                43754
text         هذا ما قد تبدو عليه دولة فلسطين المستقبلية بعا...
geo                                                        NaN
mentions                                                   NaN
hashtags                                                   NaN
id                                                 1,22222E+18
permalink    https://twitter.com/realDonaldTrump/status/122...
Name: 142, dtype: object

## Solution - Problem - Language

### langdetect API

https://pypi.org/project/langdetect/

In [22]:
# import
from langdetect import detect

In [30]:
language_test_string1 = data_test.iloc[142,:].text
language_test_string2 = data_test.iloc[144,:].text
print(language_test_string1)
print(detect(language_test_string1))
print(language_test_string2)
print(detect(language_test_string2))

if detect(language_test_string1) != "en":
    print("Problem solved")

هذا ما قد تبدو عليه دولة فلسطين المستقبلية بعاصمة في أجزاء من القدس الشرقية. pic.twitter.com/CFuYwwjSso
ar
תמיד אעמוד לצד מדינת ישראל והעם היהודי. אני תומך בחוזקה בבטיחותם ובטחונם ובזכותם לחיות במולדתם ההיסטורית. הגיע הזמן לשלום! pic.twitter.com/AtNnQtnGZs
he
Problem solved


### Amount of not english tweets


In [67]:
amount_english_tweets = 0
amount_non_english_tweets = 0
index_non_english_tweets = []

for index in range(len(data_test.text)):
    if(detect(data_test.text[index]) != "en"):
        amount_non_english_tweets = amount_non_english_tweets +1
        index_non_english_tweets.append(index)
    else:
        amount_english_tweets = amount_english_tweets +1 

English: 194
Other: 6


In [75]:
print("English:", amount_english_tweets)
print("Other:", amount_non_english_tweets)
print("Index other:", index_non_english_tweets)
for i in index_non_english_tweets:
    print(i, detect(data_test.text[i]), data_test.text[i])
    print("---")

English: 194
Other: 6
Index other: [60, 61, 142, 144, 145, 189]
60 de KEEP AMERICA GREAT!
---
61 so MAKE AMERICA GREAT AGAIN!
---
142 ar هذا ما قد تبدو عليه دولة فلسطين المستقبلية بعاصمة في أجزاء من القدس الشرقية. pic.twitter.com/CFuYwwjSso
---
144 he תמיד אעמוד לצד מדינת ישראל והעם היהודי. אני תומך בחוזקה בבטיחותם ובטחונם ובזכותם לחיות במולדתם ההיסטורית. הגיע הזמן לשלום! pic.twitter.com/AtNnQtnGZs
---
145 pl https://www. pscp.tv/w/cQBvvDFvTlFs TFJub1dwUXd8MW1ueGVReUVyVk54WA1zKet_7z8vzWFt8Te_Oj1SHWia5RzDd-o1zJWDHKC7 …
---
189 fa وزیر امور خارجه ایران می گوید ایران خواستار مذاکره با ایالات متحده است اما می خواهد که تحریم ها برداشته شود. @FoxNews @OANN نه، مرسی! https:// twitter.com/realDonaldTrum p/status/1221225245220265985 …
---


## Preprocessing


Clean the text data

- Languages other than englisch
- Links 
-- at the *beginning* of the tweet *or* at the *end*
-- removed at the beginning
- Pictures

Don´t clean

- Hashtags
- tokenizing
- stop words
- lemma
- non-alphabetical characters

In [28]:
text = data.text

In [29]:
text[0]

'A great coach and a fantastic guy. His endorsement of me in Indiana was a very big deal! https:// twitter.com/kyle__boone/st atus/1226234981808250880 …'

In [20]:
for text in data_test.text:
    print(text)
    print("--")

A great coach and a fantastic guy. His endorsement of me in Indiana was a very big deal! https:// twitter.com/kyle__boone/st atus/1226234981808250880 …
--
Pete Rose played Major League Baseball for 24 seasons, from 1963-1986, and had more hits, 4,256, than any other player (by a wide margin). He gambled, but only on his own team winning, and paid a decades long price. GET PETE ROSE INTO THE BASEBALL HALL OF FAME. It’s Time!
--
Total and complete Endorsement for Debbie Lesko! @RepDLesko Love Arizona. https:// twitter.com/repdlesko/stat us/1225484090754899969 …
--
Governor Cuomo wanted to see me this weekend. He just canceled. Very hard to work with New York - So stupid. All they do is sue me all the time! https:// twitter.com/RepStefanik/st atus/1225494053913079808 …
--
We will not be touching your Social Security or Medicare in Fiscal 2021 Budget. Only the Democrats will destroy them by destroying our Country’s greatest ever Economy!
--
...the worst weeks ever.” She could have had a mu

In [21]:
# pre processing
# remove:
# - pictures (pic.twitter.com/KSVkKL76NM) - all (beginning and end)
# - link (https://...) - all  (beginning and end)
# - not english tweets (Arabic & Hebräisch)

In [22]:
import spacy

In [23]:
nlp = spacy.load("en_core_web_sm", disable=["tagger", "parser", "entityrecognizer"])

In [ ]:
def preprocess(text):
    doc = nlp(text)
    
    # init an list to save the return
    result = []

    # is alpha: 
    # - Is the token an alpha character?
    
    for token in doc:
        # take just token without stop words & just alpha character 
        if token.is_stop == False and token.is_alpha == True: 
                result.append(token.lemma_)
    return result

In [38]:
# import library which is for cleaning 
import preprocessor as p
import re

In [54]:
text[2]

'Total and complete Endorsement for Debbie Lesko! @RepDLesko Love Arizona. https:// twitter.com/repdlesko/stat us/1225484090754899969 …'

In [55]:
clean_test = p.clean(text[2])
clean_test

'Total and complete Endorsement for Debbie Lesko! Love Arizona. https:// us/1225484090754899969'

In [56]:
clean_test2 = p.parse(text[2])
clean_test2.urls

[(83:109) => twitter.com/repdlesko/stat]

In [59]:
# TEXT CLENAING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

In [60]:
re.sub(TEXT_CLEANING_RE, ' ', str(text[2]).lower())

'total and complete endorsement for debbie lesko repdlesko love arizona   twitter com repdlesko stat us 1225484090754899969 '

In [ ]:
regex_str = [
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
]

In [49]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [46]:
str(text[0])

'A great coach and a fantastic guy. His endorsement of me in Indiana was a very big deal! https:// twitter.com/kyle__boone/st atus/1226234981808250880 …'

In [48]:
test = re.sub('https?://\S+|www\.\S+', '', text[0])
print(test)

A great coach and a fantastic guy. His endorsement of me in Indiana was a very big deal! https:// twitter.com/kyle__boone/st atus/1226234981808250880 …


In [50]:
clean_text(text[0])

NameError: name 'string' is not defined

## Make emotional classification

In [ ]:
# create data frame 
data_new = create_emotion(data)